In [11]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# Cargar archivos desde la carpeta exportada
rutas = pd.read_csv("exported_data/rutas.csv")
bases = pd.read_csv("exported_data/bases.csv")
obstaculos = pd.read_csv("exported_data/obstaculos.csv")
urgencias = pd.read_csv("exported_data/urgencias.csv")

# Dimensiones del grid
max_row = int(max(rutas['row'].max(), bases['row'].max(), obstaculos['row'].max(), urgencias['row'].max()) + 1)
max_col = int(max(rutas['col'].max(), bases['col'].max(), obstaculos['col'].max(), urgencias['col'].max()) + 1)

# Ticks y drones
ticks = int(rutas['tick'].max()) + 1
num_drones = rutas['dron'].nunique()

# Crear figura
fig, ax = plt.subplots(figsize=(8, 8))
ax.set_xlim(-0.5, max_col - 0.5)
ax.set_ylim(max_row - 0.5, -0.5)

# Grid que marca las celdas (líneas entre celdas)
ax.set_xticks([x - 0.5 for x in range(max_col + 1)], minor=False)
ax.set_yticks([y - 0.5 for y in range(max_row + 1)], minor=False)
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.grid(True, which='major', color='gray', linewidth=0.5)

# Artistas - ahora sin sumar 0.5
colors = plt.cm.tab10(np.arange(num_drones))
drone_plots = [ax.plot([], [], 'o', color=colors[i], markersize=10, label=f'Dron {i}')[0] for i in range(num_drones)]

# Obstáculos y bases en el centro de las celdas
ax.scatter(obstaculos['col'], obstaculos['row'], marker='X', color='black', s=100, label='Obstáculo', zorder=3)
ax.scatter(bases['col'], bases['row'], marker='s', color='blue', s=100, label='Base', zorder=3)

# Urgencia base (peso por celda)
urgencia_grid = np.zeros((max_row, max_col), dtype=float)
for _, row in urgencias.iterrows():
    urgencia_grid[int(row['row']), int(row['col'])] = float(row['peso'])

# Acumulado por tick: suma peso cada tick y reset a 0 cuando un dron visita la celda
acumulados = []
acum_grid = np.zeros_like(urgencia_grid, dtype=float)
for tick in range(ticks):
    acum_grid += urgencia_grid
    data_tick = rutas[rutas['tick'] == tick]
    for _, r in data_tick.iterrows():
        acum_grid[int(r['row']), int(r['col'])] = 0.0
    acumulados.append(acum_grid.copy())

vmax = max((grid.max() for grid in acumulados), default=1.0)
vmax = vmax if vmax > 0 else 1.0

# Heatmap dinámico del acumulado
im = ax.imshow(acumulados[0], cmap='Reds', origin='lower',
              extent=(-0.5, max_col - 0.5, -0.5, max_row - 0.5),
              alpha=0.6, zorder=1, vmin=0, vmax=vmax)
cbar = fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
cbar.set_label('Acumulado de urgencia')

# Función de actualización

def update(tick):
    data_tick = rutas[rutas['tick'] == tick]
    for i, plot in enumerate(drone_plots):
        dron_data = data_tick[data_tick['dron'] == i]
        if not dron_data.empty:
            plot.set_data(dron_data['col'], dron_data['row'])
        else:
            plot.set_data([], [])
    im.set_data(acumulados[tick])
    ax.set_title(f"Tick {tick}")
    return [im, *drone_plots]

ani = animation.FuncAnimation(fig, update, frames=ticks, interval=300, blit=True)
plt.legend(loc='upper right')
plt.close()
HTML(ani.to_jshtml())
